COMP 215 - LAB 6
----------------
#### Name(s):
#### Date:

By the end of this lab you should be able to:
  * create a Watts-Strogatz graph both from scratch and from the Networkx module
  * measure the average clustering coefficient and path length of a network
  * visualize summative data of a graph


During this lab, you will be introduced to the following:
  * numpy arrays
  * local file i/o in Google Colab

(this lab is based on workbooks provided in Allen Downey's 'Think Complexity')

## Social Networking

This lab uses graphs to explore social networks using Facebook data.  In this lab you will create a simulated model of the network using a Watts-Strogatz graph and compare some properties of the simulated network and the real Facebook network.

In [ ]:
# put your imports here

In [1]:
import numpy as np
import networkx as nx
import random
from collections import deque

## Get the Facebook Data

Stanford Network Analysis Platform (SNAP) is a general purpose network analysis and graph mining library.  In previous labs, we have used APIs to access data.  For this lab, we will copy the data to a local file.  Download the ```facebook_combined.txt.gz``` file from [SNAP](https://snap.stanford.edu/data/egonets-Facebook.html), unzip it, and copy ```facebook_combined.txt``` to the ```Files``` folder in Colab.  

Look at the content of the file and read the SNAP webpage to understand what the data represents before moving on to the rest of the lab.

## Exercise 1: Make Facebook Graph

Write a function that reads the file, one edge per line, specified by the two integer node IDs given in each line of the file and returns a ```networkx``` graph representing the data.  You can do this with Python's built-in file handling, or you could use ```numpy```'s ```loadtxt``` function.  Write a unit test to check that the network has 4039 nodes and 88234 edges (as given in the Dataset Statistics on the SNAP site) and draw the Facebook network (this takes about a minute).


In [11]:
# Ex 1 code here
def load_facebook_graph(filename):
    G = nx.Graph()
    with open(filename, "r") as file:
        for line in file:
            node1, node2 = map(int, line.strip().split())
            G.add_edge(node1, node2)
    return G

facebook_file_path = "facebook_combined.txt"
facebook_graph = load_facebook_graph(facebook_file_path)
print("Facebook graph loaded:")
print("  Nodes:", facebook_graph.number_of_nodes())
print("  Edges:", facebook_graph.number_of_edges())

Facebook graph loaded:
  Nodes: 4039
  Edges: 88234


## Exercise 2: Clustering Coefficients

With larger graphs, it can take a long time to compute clustering coefficients and path lengths. We can estimate them by sampling without much loss of accuracy if the sample size is large enough.  Write a function that calculates the average clustering coeffient for a random subset of a N nodes in a network.  You may use the ```node_clustering``` and ```all_pairs``` functions from Chapter 5 of the textbook.  You may also use the ```numpy``` module to calculate the mean; note that there is a ```nanmean``` function.

Check that your clustering coeffients function gives a similar answer to the ```networkx``` ```average_clustering``` function.


In [12]:
# Ex 2 code here
def average_clustering_subset(G, sample_size=1000):
    nodes = list(G.nodes)
    sample_nodes = random.sample(nodes, min(sample_size, len(nodes)))
    clustering_values = [nx.clustering(G, node) for node in sample_nodes]
    return np.nanmean(clustering_values)

clustering_estimate = average_clustering_subset(facebook_graph, sample_size=1000)
clustering_exact = nx.average_clustering(facebook_graph)
print("\nClustering Coefficients:")
print("  Estimated (sampled):", clustering_estimate)
print("  Exact (networkx):", clustering_exact)



Clustering Coefficients:
  Estimated (sampled): 0.6084561348881148
  Exact (networkx): 0.6055467186200876


## Exercise 3: Average Shortest Path Length

Write a function that calculates the average shortest path length for all pairs of nodes in a network.  You may use the ```shortest_path_dijkstra``` function from Chapter 5 of the textbook.  Using that function, it took my algorithm about 2 minutes to find the average shortest path over all pairs of nodes.


Check that your average shortest path length function gives a similar answer to the ```networkx``` ```average_shortest_path_length``` function.


In [13]:
# Ex 3 code here
def average_shortest_path_sampled(G, sample_size=500):
    nodes = list(G.nodes)
    sampled_nodes = random.sample(nodes, min(sample_size, len(nodes)))
    path_lengths = []
    for node in sampled_nodes:
        lengths = nx.single_source_shortest_path_length(G, node)
        path_lengths.extend(lengths.values())
    return np.mean(path_lengths)

path_length_estimate = average_shortest_path_sampled(facebook_graph, sample_size=500)
print("\nEstimated Average Shortest Path Length (sampled):", path_length_estimate)



Estimated Average Shortest Path Length (sampled): 3.6864822975984155


Here is a function from the textbook that takes a sample of path lengths to estimate the average shortest path length.  You may use this in the rest of the lab so that you don't need to wait for the whole full averaging algorithms above to run.

In [ ]:

def sample_path_lengths(G, nodes=None, trials=100):
    """Choose random pairs of nodes and compute the path length between them.
    G: Graph
    N: number of pairs to choose
    returns: list of path lengths
    """
    if nodes is None:
        nodes = list(G)
    else:
        nodes = list(nodes)

    pairs = np.random.choice(nodes, (trials, 2))
    lengths = [nx.shortest_path_length(G, *pair)
               for pair in pairs]
    return lengths

def estimate_path_length(G, nodes=None, trials=1000):
    return np.mean(sample_path_lengths(G, nodes, trials))

In the exercises above, you should have found that the Facebook network has an average clustering coefficient around 0.6 and an average shortest path length of around 3.7. Note that this corresponds to a 'degree of separation' of less than 6.   

## Exercise 4: WS Graph

Construct a WS graph with the same number of nodes and average degree as the Facebook network using the ```make_ws_graph``` function from Chapter 5.  Find the value of p (probability of rewire) that reproduces a clustering coefficient and average shortest path length of the Facebook network.  (Note that there is a ```nx.watts_strogatz_graph``` that you may use after you have demonstrated that you can create a WS graph using the functions from Chapter 5.).

What could this value of p tell you about the actual social network that this Facebook data represents?  (Think about what p means in the model and what that would represent in the data.)

In [14]:
# Ex 4 code here
def construct_ws_graph(n, k, p_values, target_clustering, target_path_length):
    best_p = None
    best_score = float("inf")
    best_clustering = None
    best_path_length = None

    for p in p_values:
        ws_graph = nx.watts_strogatz_graph(n, k, p)
        clustering = nx.average_clustering(ws_graph)
        path_length = nx.average_shortest_path_length(ws_graph)
        # We combine the differences in clustering and path length into a score.
        score = abs(clustering - target_clustering) + abs(path_length - target_path_length)
        if score < best_score:
            best_score = score
            best_p = p
            best_clustering = clustering
            best_path_length = path_length
    return best_p, best_clustering, best_path_length

avg_degree = int(np.mean([deg for _, deg in facebook_graph.degree()]))
print("\nAverage degree of graph:", avg_degree)

p_values = np.linspace(0, 1, 10)

best_p, ws_clustering, ws_path_length = construct_ws_graph(
    n=facebook_graph.number_of_nodes(),
    k=avg_degree,
    p_values=p_values,
    target_clustering=clustering_exact,
    target_path_length=path_length_estimate
)
print("\nWatts-Strogatz Model Tuning:")
print("  Best p value:", best_p)
print("  WS Graph Clustering Coefficient:", ws_clustering)
print("  WS Graph Average Shortest Path Length:", ws_path_length)


Average degree of graph: 43

Watts-Strogatz Model Tuning:
  Best p value: 0.1111111111111111
  WS Graph Clustering Coefficient: 0.5179376300581644
  WS Graph Average Shortest Path Length: 2.974095191987091
